In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import fuzzywuzzy
from fuzzywuzzy import fuzz
from  nltk import word_tokenize
import torch.optim as optim
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import io

D:\Softwares\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [3]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [4]:
def get_processed_tokens_for_english_tweets():
    f = "english/agr_en_train.csv"
    # preprocessing english tweets.
    #ingesting english csv file
    df = pd.read_csv(f,names = ['source','comment','annotation'],encoding='UTF-8')
    df['comment'] = df.comment.str.strip()   # removing spaces
    comments = np.asarray(df['comment'])    # dividing the dataframe into comments and tags and converting to array
    tags = np.asarray(df['annotation'])
    print((len(comments)))
    print(len(tags))
    stop_words = set(stopwords.words('english'))  #english stop words list
    processed_tokens = []
    for comment in comments:
    # comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        tokenizer = nltk.RegexpTokenizer(r"\w+")   # removal of punctuation and tokenizing
        new_words = tokenizer.tokenize(URL_REMOVAL)
        sentence = []
        for word in new_words:
            if word not in stop_words:           #checking for stop words on each sentence
                sentence.append(word)
        processed_tokens.append(sentence)
    return processed_tokens

In [5]:
#-----------------For hinglish dataset

def get_processed_hindi_tokens():
    Hindi_text  = "hindi/agr_hi_dev.csv"
    df1 = pd.read_csv(Hindi_text,names = ['source','comment','annotation'],encoding='UTF-8')
    df1['comment'] = df1.comment.str.strip()   # removing spaces
    hindi_comments = np.asarray(df1['comment'])    # dividing the dataframe into comments and tags and converting to array
    hindi_tags = np.asarray(df1['annotation'])
    print((hindi_comments[1])) 
    processed_Hindi_tokens = []
    for comment in hindi_comments:
    #   comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        Emoji_removal = remove_emoji(URL_REMOVAL)
        if (isEnglish(Emoji_removal) == True):
            Emoji_removal = re.sub(r'[^\w\s]','',Emoji_removal)# removal of punctuation and tokenizing
        processed_Hindi_tokens.append(word_tokenize(Emoji_removal))
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens


In [6]:
#-----------Transliteration and translation
def get_transliteration_Hinglish_Hindi_dict():
    transliteration_dict = "transliterations.hi-en.csv"
    t_dict = pd.read_csv(transliteration_dict,names = ['Hinglish','Hindi'],encoding='UTF-8',sep='\t')
    t_dict['Hinglish'] = t_dict['Hinglish'].str.strip()
    t_dict['Hindi'] = t_dict['Hindi'].str.strip()
    print(t_dict)
    t_dict = np.asarray(t_dict)
    print("After NP array")
    print(t_dict)
    return t_dict

#--------------profanity dictionary
def get_profanity_dict():
    profanity_dict = "ProfanityText.txt"
    P_dict = pd.read_csv(profanity_dict,names = ['Hinglish','English'],encoding='UTF-8',sep='\t')
    P_dict['Hinglish'] = P_dict['Hinglish'].str.strip()
    P_dict['English'] = P_dict['English'].str.strip()
    print("Profanity")
    print(P_dict)
    P_dict = np.asarray(P_dict)
    print("After NP array")
    print(P_dict)
    return P_dict

In [7]:
#------------------Translation of hindi text back to english-------
def translate_hindi_to_english():
    Hindi_dict = "Hindi_English_Dict.csv"
    H_dict = pd.read_csv(Hindi_dict,names = ['Hindi','English'],encoding='UTF-8')

    HE_dict_F = "HE_dictionary_functions.csv"
    H_dict_F = pd.read_csv(HE_dict_F,names = ['Hindi','English'],encoding='UTF-8')
    H_dict_F['Hindi'] = H_dict_F['Hindi'].str.strip()
    H_dict_F['English'] = H_dict_F['English'].str.strip()

    H_hindi_F = np.asarray(H_dict_F['Hindi'])
    H_english_F = np.asarray(H_dict_F['English'])

    H_dict['Hindi'] = H_dict['Hindi'].str.strip()
    H_dict['English'] = H_dict['English'].str.strip()

    H_hindi = np.asarray(H_dict['Hindi'])
    H_english = np.asarray(H_dict['English'])
    
    HE_dict = dict(zip(H_hindi,H_english))
    H_dict_F = dict(zip(H_hindi_F,H_english_F))

    EH_dict = {v:k for k, v in HE_dict.items()}
    EH_dict_F = {v:k for k, v in H_dict_F.items()}
    
    return HE_dict, H_dict_F, EH_dict, EH_dict_F

In [8]:
# Convert
def get_translated_hindi_english(HE_dict, H_dict_F):
    for i in range(0,len(processed_Hindi_tokens)):
        #print(i)
        for j in range (0,len(processed_Hindi_tokens[i])):
            Str = processed_Hindi_tokens[i][j]
            if(Str in HE_dict):
                processed_Hindi_tokens[i][j] = HE_dict[Str]
            elif(Str in H_dict_F):
                processed_Hindi_tokens[i][j] = H_dict_F[Str]
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens

In [9]:
def get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict):
    for i in range(0,len(processed_Hindi_tokens)):
        if i == 50:
            break
        for j in range (0,len(processed_Hindi_tokens[i])):
            flag = 0
            Str1 = (processed_Hindi_tokens[i][j])
            max_ratio = 60
            max_ratio_P = 75   #needs to be adjusted
            if (Str1 in EH_dict): # check whether the values exists in english dictionary or not.
                continue
            for l in range(0,len(P_dict)):
                Str2 = P_dict[l][0]
                Ratiostr1 = fuzz.ratio(Str1,Str2)
                if (Ratiostr1 >= max_ratio_P):
                    print(Ratiostr1)
                    max_ratio_P = Ratiostr1
                    flag = 1
                    print(processed_Hindi_tokens[i][j])
                    processed_Hindi_tokens[i][j] = P_dict[l][1]
                    print(f"{flag}-{processed_Hindi_tokens[i][j]}") 
                    break
            for p in EH_dict_F:
                Ratiostr1 = fuzz.ratio(Str1,str(p))
                if(Ratiostr1 >= 98):
                    flag = 1
                    break
            if (flag == 1):
                continue
            else:
                for k in range(0,len(t_dict)):
                    Str2 = t_dict[k][0]
                    Ratiostr1 = fuzz.ratio(Str1,Str2)
                    if (Ratiostr1 > max_ratio):
                        max_ratio = Ratiostr1
                        processed_Hindi_tokens[i][j] = t_dict[k][1]
    print(processed_Hindi_tokens[0])
    print(processed_Hindi_tokens[1])
    print(processed_tokens[12])
    return processed_Hindi_tokens

In [27]:
HE_dict.values()

dict_values(['xi', '12', '130', 'lobby', '1500', '155', '15', '1865', '186', '189', '18th', '1981', '1998', '1999', '1', '2000', '2001', '2002', '2003', '2004', '20', 'twenty p', '22', '23', '24', '25', '26', '29', '29th', '2', '2p', '35', '39', '3', 'tertiary', '400', '45', '4', '500', '58', '5', 'five p', '68', '71', '7th', '80', '83', '98', 'stellar', 'ace', 'AA', 'newborn', 'shock', 'backseat driver', 'abacus', 'a bad dream', 'fiend', 'a bad sailor', 'a bag of bones', 'a bag of nerves', 'xiii', 'a ball of fire', 'a ballpark-figure', 'abandoned', 'abandon', 'devolve', 'throw overboard', 'abnegate', 'spike', 'boycott', 'wantonness', 'abandonment', 'a baptism of fire', 'mortify', 'diss', 'snub', 'embarrassed', 'slack off', 'waning', 'prioress', 'priory', 'archimandrite', 'reduce', 'abstraction', 'abbreviation', 'ABCs', 'alphabet', 'renounce', 'stepping down', 'belly', 'belly', 'abdominal', 'shanghai', 'hijacking', 'snug', 'Aberdeen', 'wrongful', 'aberration', 'join', 'advance', 'stopp

In [10]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [30]:
def load_vectors(fname):
    path = str(fname)
    fin = io.open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [11]:
processed_tokens = get_processed_tokens_for_english_tweets()
processed_Hindi_tokens = get_processed_hindi_tokens()
t_dict = get_transliteration_Hinglish_Hindi_dict()
P_dict = get_profanity_dict()
HE_dict, H_dict_F, EH_dict, EH_dict_F = translate_hindi_to_english()
processed_Hindi_tokens = get_translated_hindi_english(HE_dict, H_dict_F)
processed_Hindi_tokens = get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict)

11999
11999
First stage par dus jootey khaye Grover  se
['randtv', 'tumhare', 'najayaz', 'baap', 'is', 'area', 'hai', 'ki', 'waha', 'koi', 'nahi', 'has', 'sakta', 'haraami', 'azad', 'mulk', 'hai', 'sab', 'jagah', 'jayenge']
           Hinglish        Hindi
0         hajagiree       हजगिरी
1          chekaanv        चेकॉव
2        spinagaarn   स्पिनगार्न
3             medal         मेडल
4       chetthinaad    चेत्तिनाद
...             ...          ...
14914          roda         रोडा
14915  shymaleshwor  स्यामलेश्वर
14916           bar          वार
14917       leonard    लियोनार्ड
14918      gurudwar   गुरूद्वारा

[14919 rows x 2 columns]
After NP array
[['hajagiree' 'हजगिरी']
 ['chekaanv' 'चेकॉव']
 ['spinagaarn' 'स्पिनगार्न']
 ...
 ['bar' 'वार']
 ['leonard' 'लियोनार्ड']
 ['gurudwar' 'गुरूद्वारा']]
Profanity
       Hinglish         English
0         badir           idiot
1    badirchand           idiot
2       bakland           idiot
3        bhadva            pimp
4     bhootnika  son 

In [24]:
processed_Hindi_tokens[4]

['beta',
 'to',
 'टू',
 'अपने',
 'बाप',
 'का',
 'है',
 'permission',
 'to',
 'टू',
 'मान',
 'ही',
 'लीयर',
 'ab',
 'मुकद्दर',
 'mat',
 'सामरा',
 'यही',
 'breed',
 'टोबॅगो',
 'की',
 'बाज़ी',
 'nigger',
 'जाएगी',
 'नहीं']

In [24]:
class MIMCT(nn.Module):   
    def __init__(self,input_channel,output_channel,embedding_dim,hidden_dim,kernel_size,feature_linear, word_vector_list):
        super(MIMCT, self).__init__()
        
        #Primary Input
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data.copy_(torch.from_numpy(word_vector_list))
        
        #Secondary Input (Profanity Vector, LiWC and Sentiment Score)
        
        
        self.CNN_Layers = nn.Sequential( 
            nn.Conv1d(input_channel, output_channel,kernel_size[0], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[1], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[2], stride=1),
            nn.Flatten(),nn.Dropout(p=0.25),
            nn.Linear(feature_linear, 3),
            nn.Softmax()
            )
        
        
        #create LSTM.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim,3)
        self.dropout = nn.Dropout(p=0.20)
        self.softmax = nn.Softmax()
        
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2)
        self.linear = nn.Linear(input_channel+1,3)
    def forward(self,x):
        
        #Concatenation of Embeddings
        
        cnn_output = self.CNN_Layers(x)
      #  y = self.LSTM_Layers(x)
        lstm_out, _ = self.lstm(x)
        lstm_out= self.dropout(lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(lstm_out.size(1), -1))
        lstm_output = F.log_softmax(tag_space, dim=1)
        #concat the outputs the compile layer with categorical cross-entropy the loss function,
        lstm_output = lstm_output.view(lstm_output.size(0),-1)
        cnn_output = cnn_output.view(cnn_output.size(0),-1)
        
        X = torch.cat((lstm_output,cnn_output))
        X = X.view(1,X.size(0),X.size(1))
        X = self.maxpool(X)
        
        X = self.linear(X.view(X.size(2), -1))
        X = self.softmax(X)
        print(X)
        return x

In [25]:
#try with output channel 1 as well.
batch_size = 1
input_channel = 16
embedding_dim = 200
output_channel = 16
kernel_size = [20,15,10]
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel

In [ ]:
#Getting English Dataset
data_vector_English = load_vectors("cc.en.300.vec")

In [ ]:
word_vector_list = []
for i, word in enumerate(data_vector_English):
    word_to_ix[word] = len(word_to_ix)
    word_vector_list.append(list(data_vector[word]))

In [26]:
#Parameters for LSTM
hidden_dim = 64
dropout = 0.25, 
#recurrent_dropout = 0.3

#The length vocabulary formed after translating the Hindi and Hinglish tokens to English along with English Tokens 
vocab_size = len(set(HE_dict.values())) + len(set(H_dict_F.values())) + len(set(EH_dict.keys())) + len(set(EH_dict_F.keys()))

model = MIMCT(input_channel, output_channel, embedding_dim, hidden_dim, kernel_size, feature_linear)

#Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

#output = model(input1)
#create the loss criterion and training loops
input1 = torch.randn(batch_size,input_channel,embedding_dim)
print(input1.shape)
output = model(input1)

torch.Size([1, 16, 200])


D:\Softwares\Anaconda\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.2235, 0.4550, 0.3215]], grad_fn=<SoftmaxBackward>)


<ipython-input-24-cfeef8848fde>:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.softmax(X)


In [ ]:
'''
m = nn.Conv1d(1, 2,1,stride=2)
input1 = torch.randn(10)
cnn1d_2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=2)
input1 = input1.unsqueeze(0).unsqueeze(0)
input1.shape
cnn1d_2(input1)



#------------CNN_Class----------------#
kernel_size = [4,3,2]
embedding_dim = 10
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1

Layer1 = nn.Conv1d(16, 16, 4, stride=1)
Layer2 = nn.Conv1d(16, 16, 3, stride=1)
Layer3 = nn.Conv1d(16, 16, 2, stride=1)
Dropout_layer = nn.Dropout(p=0.25)
dense_layer = nn.Linear(Feature_layer3, 3)
nn.Flatten

#------------Forward------------------#
batch_size = 1
input_channel = 16
embedding_dim = 10
input1 = torch.randn(batch_size,input_channel,embedding_dim)
output = Layer1(input1)
output1 = Layer2(output)
output2 = Layer3(output1)
output3 = Dropout_layer(output2)
output4 = dense_layer(output3)
nn.flatten



x = torch.tensor([[1.0, -1.0],[0.0,  1.0],[0.0,  0.0]])

in_features = x.shape[1]  # = 2
out_features = 2

m = nn.Linear(in_features, out_features)
m.weight

class MIMCT(nn.Module):   
    def __init__(self,input_channel,output_channel,embedding_dim,kernel_size,feature_linear):
        super(MIMCT, self).__init__()
        self.CNN_Layers = nn.Sequential( nn.Conv1d(input_channel, output_channel,kernel_size[0], stride=1),nn.Conv1d(input_channel, output_channel, kernel_size[1], stride=1),nn.Conv1d(input_channel, output_channel, kernel_size[2], stride=1),nn.Flatten(),nn.Dropout(p=0.25),nn.Linear(feature_linear, 3),nn.Softmax())

    def forward(self,x):
        x = self.CNN_Layers(x)
        return x

#try with output channel 1 as well.
batch_size = 1
input_channel = 16
embedding_dim = 200
output_channel = 16
kernel_size = [20,15,10]
embedding_dim = 200
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel
model = MIMCT(input_channel,output_channel,embedding_dim,kernel_size,feature_linear)

output = model(input1)
'''




''''
kernel_size = [20,15,10]
embedding_dim = 200
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel
m = nn.Sequential( nn.Conv1d(input_channel,16,kernel_size[0], stride=1),nn.Conv1d(input_channel, output_channel, kernel_size[1], stride=1),nn.Conv1d(input_channel, output_channel, kernel_size[2], stride=1),nn.Flatten(),nn.Dropout(p=0.25),nn.Linear(feature_linear, 3),nn.Softmax())

#------------Forward------------------#
batch_size = 1
input_channel = 16
embedding_dim = 200
input1 = torch.randn(batch_size,input_channel,embedding_dim)
test_input = input1
output = m(test_input)
nn.flatten
'''

In [ ]:
a = torch.randn(16, 3)
b = torch.randn(1, 3)

In [ ]:
a, b

In [ ]:
a.mean()

In [ ]:
torch.stack(a,b)

In [ ]:
torch.stack((a,b))